# MVP - Qualidade de Software, Segurança e Sistemas Inteligentes - Bruno Rubén Favorito Balbuena

#### O objetivo deste *notebook* é treinar um modelo de *machine learning* em cima da base de dados  "[Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic)", que deverá  prever se um passageiro teria sobrevivido ao naufrágio do Titanic, com base em atributos a serem descritos logo abaixo. O foco está na construção de um *pipeline* completo de classificação, incluindo pré-processamento, avaliação com validação cruzada, ajuste de hiperparâmetros e comparação entre algoritmos.

####Dicionário de Dados

| **Atributo** | **Definição**                                | **Valor**                                      |
| ------------ | -------------------------------------------- | ---------------------------------------------- |
| Survived     | Sobrevivência do passageiro(a)               | 0 = Não, 1 = Sim                               |
| Name         | Nome do passageiro(a)                        | —                                              |
| PassengerId  | ID do passageiro(a)                          | —                                              |
| Pclass       | Classe do ticket                             | 1 = 1ª, 2 = 2ª, 3 = 3ª                         |
| Sex          | Gênero                                       | —                                              |
| Age          | Idade (anos)                                 | —                                              |
| SibSp        | Nº de irmãos(ãs)/cônjuges a bordo do Titanic | —                                              |
| Parch        | Nº de pais/filhos(as) a bordo do Titanic     | —                                              |
| Ticket       | Número do ticket                             | —                                              |
| Fare         | Tarifa paga pelo passageiro(a)               | —                                              |
| Cabin        | Número da cabine                             | —                                              |
| Embarked     | Porto de embarque                            | C = Cherbourg, Q = Queenstown, S = Southampton |


<ul>
<li> Pclass: Um proxy para status socioeconômico </li>
<ul>
<li> 1ª = Alta </li>
<li> 2ª = Média </li>
<li> 3ª = Baixa </li>
</ul>


<li>Age: Idade é uma fração se for menor que 1. Se a idade for estimada, será na formato xx.5</li>

<li>SibSp: A base de dados define relações familiares da seguinte forma: </li>
Irmãos(ãs): irmão, irmã, meio-irmão, meia-irmã <br />
Cônjuge: esposo, esposa (amantes e noivos não são levados em consideração)
<li>Parch: A base de dados define relações familiares da seguinte forma:.</li>
Pais: mãe, pai <br />
Filhos(as): filha, filho, enteado, enteada <br />
Algumas crianças viajaram somente com uma babá, então parch=0 para esses casos.
</ul>

## Importação de bibliotecas

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

## Constantes globais

* SEED: Declarei a semente de aleatoriedade como uma variável global para garantir a reprodutibilidade dos resultados, especialmente no embaralhamento dos dados, ao executar o código múltiplas vezes.
* TEST_SIZE: Defini a proporção do conjunto de teste como 20%, adotando a divisão comum 80/20 utilizada no método *Holdout*.
* N_SPLITS: Número de divisões (*folds*) a serem utilizadas na validação cruzada estratificada. O valor foi escolhido por ser um dos padrões mais comuns e equilibrar bem entre viés e variância.
* SCORING: Métrica utilizada para avaliar a performance do modelo. Como se trata de um problema de classificação, optei por uma das métricas mais comuns e interpretáveis nesse tipo de tarefa.

In [ ]:
SEED = 1
TEST_SIZE = 0.20
N_SPLITS = 10
SCORING = 'accuracy'

## Carga da base de dados

Realizei o download do arquivo **train.csv** e o carreguei em um repositório do GitHub. Utilizando a biblioteca **pandas**, fiz a leitura do arquivo e imprimi informações básicas sobre a base de dados. Essa base contém 891 tuplas e 12 atributos, conforme descrito no dicionário de dados.

In [ ]:
datasetUrl = "https://raw.githubusercontent.com/Bansuk/titanic-survival-prediction-back-end/refs/heads/main/src/data/train.csv"
dataframe = pd.read_csv(datasetUrl)
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Tratamento dos atributos e dados ausentes

Realizei o tratamento de dados ausentes e não-númericos, tornando-os adequados como *input* para os algoritmos do **scikit-learn**, com base em uma cópia do **DataFrame** original.

* Preenchimento de dados ausentes
<ul>
  <li><b>Age</b>: Substitui os valores vazios pela mediana da idade com base em atributos correlacionados dentro desse contexto de dados. (Como ainda restaram valores <b>NaN</b> ao utilizar todos os atributos abaixo, adicionei uma segunda imputação com um grupo reduzido de atributos.)</li>
  
  <ul>
    <li><b>Sex</b>: O sexo do passageiro(a) pode influenciar sua idade média, especialmente considerando papéis sociais históricos;</li>
    <li><b>Pclass</b>: Classes mais altas tendem a estar associadas a passageiros mais velhos e com maior poder aquisitivo;</li>
    <li><b>Title</b>: Reflete o papel social do passageiro, o que pode indicar sua idade (extração feita a partir do atributo <b>Name</b>);</li>
    <li><b>SibSp e Parch</b>: Passageiros com irmãos, cônjuges, pais ou filhos a bordo tendem a estar em faixas etárias específicas (por exemplo, ter filhos pode indicar maior idade; estar com os pais, menor idade).</li>
  </ul>
  <li><b>Embarked</b>: Há apenas duas tuplas sem a informação sobre embarque, ambas mulheres com o mesmo valor para ticket, cabine e tarifa. Como são apenas dois dados faltantes e não encontrei outros(as) passageiros(as) com valores mencionados similares, optei por preencher com a moda de <b>Embarked</b> para tickets de mesma classe dessas duas, no caso, 1ª classe.</li>
  <li><b>Cabin</b>: Como 77.1% da base não possui o valor de <b>Cabin</b> preenchido, e julgo esse atributo potencialmente relevante (por poder representar a localização do passageiro(a) no navio, o que pode ter influenciado a sobrevivência, principalmente considerando que o impacto ocorreu à noite), optei por não imputar valores arbitrários. Em vez disso, criei um novo atributo binário <b>HasCabin</b>, que indica se o passageiro possui uma cabine registrada — o que pode, por exemplo, estar associado à classe social.</li>
</ul>
* Codificação de variáveis categóricas
<ul>
  <li><b>Sex</b>: Converti os valores de texto <i>female</i> e <i>male</i> em 0 e 1, respectivamente.</li>
  <li><b>Embarked</b>: Apliquei codificação, gerando colunas binárias para cada porto de embarque. Para evitar multicolinearidade, descartei a primeira coluna gerada.</li>
</ul>
</ul>

Removi os seguintes atributos dos dados de entrada:
* **PassengerId**: Apenas representa a ordem dos dados dos passageiros(as).
* **Ticket**: Valor aleatório sem relação com os demais atributos.
* **Name**: Não possui correlação com o resultado, apenas sendo útil para extração da informação de título.
* **Cabin**: Grande maioria das tuplas está com esse valor vazio. Utilizado para criar o atributo **HasCabin**.
* **Survived**: Dado de saída, sendo o valor previsto.

In [ ]:
df = dataframe.copy()
le = LabelEncoder()

df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df['Age'] = df.groupby(['Sex', 'Pclass', 'Title', 'SibSp', 'Parch'])['Age'].transform(lambda x: x.fillna(x.median()))
df['Age'] = df.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))
age_medians = df.groupby(['Sex', 'Pclass', 'Title', 'SibSp', 'Parch'])['Age'].median()
age_medians_overall = df.groupby(['Sex', 'Pclass'])['Age'].median()
df = df.drop('Title', axis=1)
embarked_mode_pclass1 = df[df['Pclass'] == 1]['Embarked'].mode()[0]

df.loc[df['Embarked'].isnull() & (df['Pclass'] == 1), 'Embarked'] = df[df['Pclass'] == 1]['Embarked'].mode()[0]
df['HasCabin'] = df['Cabin'].notnull().astype(int)

df['Sex'] = le.fit_transform(df['Sex'])
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked', drop_first=True)

X = df.drop(columns=['PassengerId', 'Ticket', 'Name', 'Cabin', 'Survived'])
X[X.select_dtypes('bool').columns] = X.select_dtypes('bool').astype(int)
y = df['Survived']

##Aplicação do Método de *Holdout*

Realizei a separação dos dados em conjuntos de treino e teste utilizando embaralhamento para evitar viés decorrente da ordenação original. Para garantir reprodutibilidade, fixei uma semente aleatória. Além disso, apliquei estratificação com base no atributo alvo, de forma que ambos os conjuntos preservem a proporção de passageiros sobreviventes e não sobreviventes.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=TEST_SIZE, shuffle=True, random_state=SEED, stratify=y)

## Modelagem

### Criação e avaliação dos algoritmos KNN (K-vizinhos mais próximos), CART (Árvore de classificação e regressão), NB (Naive Bayes) e SVM (Máquina de vetor de suporte): sem tratamento, padronizado e normalizado

Para tratar o dilema viés x variância, utilizei a validação cruzada estratificada com embaralhamento dos dados.
<ol>
  <li>Inicializei o gerador de números aleatórios para garantir a reprodutibilidade dos dados em todas as execuções.</li>
  <li>Obtive as divisões estratificadas a serem utilizadas na validação cruzada.</li>
  <li>Instanciei 4 algoritmos de classificação (sem configurações adicionais):
  <ul>
    <li>K-vizinhos mais próximos</li>
    <li>Árvore de classificação e regressão</li>
    <li>Naive Bayes</li>
    <li>Máquinas de vetores de suporte</li>
  </ul>
  </li>
  <li>Para evitar que atributos com escalas maiores dominassem o treinamento, defini três formas de pré-processamento:
    <ul>
      <li>Valores sem tratamento (original)</li>
      <li>Valores padronizados: transformando os dados para terem média 0 e desvio padrão 1.</li>
      <li>Valores normalizados: redimensiona os dados para o intervalo entre 0 e 1.</li>
    </ul>
  </li>
  <li>Criei <i>pipelines</i> para cada combinação possível entre algoritmos e pré-processadores</li>
  <li>Para cada <i>pipeline</i>, executei a validação cruzada estratificada com 10 divisões, utilizando todos os núcleos da máquina para acelerar o processo.</li>
  <li>Organizei os <i>outputs</i> pela média dos resultados, acompanhada do desvio padrão, em ordem decrescente.</li>
</ol>

In [ ]:
np.random.seed(SEED)
kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

algorithms = {
    'KNN': KNeighborsClassifier(),
    'CART': DecisionTreeClassifier(),
    'NB': GaussianNB(),
    'SVM': SVC()
}

preprocessors = {
    'original': None,
    'standard': StandardScaler(),
    'normalization': MinMaxScaler()
}

pipelines = {}
results = {}

def set_pipelines(preprocessors, algorithms, pipelines):
  for preprocessor_name, preprocessor in preprocessors.items():
    for model_name, model in algorithms.items():
        pipeline_name = f"{model_name}-{preprocessor_name}"

        if preprocessor is None:
            pipeline_steps = [(model_name.lower(), model)]
        else:
            pipeline_steps = [
                (f"{preprocessor_name}_scaler", preprocessor),
                (model_name.lower(), model)
            ]

        pipelines[pipeline_name] = Pipeline(pipeline_steps)

set_pipelines(preprocessors, algorithms, pipelines)

for name, pipeline in pipelines.items():
      cv_results = cross_val_score(
          pipeline, X_train, y_train,
          cv=kfold, scoring=SCORING, n_jobs=-1
      )
      results[name] = cv_results

sorted_results = sorted(
    results.items(),
    key=lambda x: x[1].mean(),
    reverse=True
)

print("Avaliação dos Resultados")
print("-" * 50)
for name, scores in sorted_results:
    print(f"{name}: Média={scores.mean():.4f}, Desvio Padrão={scores.std():.4f}")

Avaliação dos Resultados
--------------------------------------------------
SVM-standard: Média=0.8062, Desvio Padrão=0.0356
SVM-normalization: Média=0.7964, Desvio Padrão=0.0532
KNN-standard: Média=0.7922, Desvio Padrão=0.0586
KNN-normalization: Média=0.7880, Desvio Padrão=0.0522
NB-original: Média=0.7781, Desvio Padrão=0.0419
NB-standard: Média=0.7781, Desvio Padrão=0.0419
NB-normalization: Média=0.7781, Desvio Padrão=0.0419
CART-normalization: Média=0.7599, Desvio Padrão=0.0529
CART-standard: Média=0.7584, Desvio Padrão=0.0345
CART-original: Média=0.7472, Desvio Padrão=0.0465
KNN-original: Média=0.7078, Desvio Padrão=0.0301
SVM-original: Média=0.6700, Desvio Padrão=0.0378


### Otimização de hiperparâmetros

Em busca de tornar o modelo ainda melhor para solucionar o problema, farei o *tuning* para o melhor algoritmo encontrado: o SVM padronizado. Essa técnica consiste em ajustar os hiperparâmetros do algoritmo para adaptá-lo ao problema em questão. Para isso, utilizarei o GridSearchCV que consiste na montagem de um *grid* com a combinação entre os hiperparâmetros fornecidos e um conjunto de valores a serem avaliados. Para cada combinação, um modelo é construído e avaliado utilizando validação cruzada.  <br />
O conjunto de hiperparâmetros para o algoritmo SVM foi composto da seguinte forma:
*   Kernel: função matemática que transforma os dados em um espaço de maior dimensionalidade. O kernel linear é simples e direto, enquanto o kernel RBF (Função de Base Radial) é mais flexível e lida melhor com dados não lineares.
*   C: controla a penalidade por classificações incorretas; quanto maior, menos tolerante o modelo é a erros no treinamento.
*   Gamma: define a influência de cada exemplo de treino (utilizado apenas no kernel RBF).
<br />

Foram definidos dois conjuntos de hiperparâmetros: um conjunto completo com 400 combinações (executado previamente) e um conjunto reduzido baseado nos melhores resultados, com apenas 10 combinações, que é o utilizado no código a seguir por questões de desempenho.

In [ ]:
np.random.seed(SEED)

pipelines = {}

algorithms = {
    'svm': SVC()
}

set_pipelines(preprocessors, algorithms, pipelines)

param_grid = {
    'svm__kernel': ['linear', 'rbf'],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

param_grid_best = {
    'svm__kernel': ['rbf'],
    'svm__C': [10],
    'svm__gamma': [0.01]
}

for name, model in pipelines.items():
    grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid_best,
    cv=kfold,
    scoring=SCORING,
    n_jobs=-1,
    verbose=1
)
    grid.fit(X_train, y_train)

    print("%s Melhor: %.4f usando %s" %
          (name, grid.best_score_, grid.best_params_))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
svm-original Melhor: 0.7625 usando {'svm__C': 10, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
svm-standard Melhor: 0.8147 usando {'svm__C': 10, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
svm-normalization Melhor: 0.7823 usando {'svm__C': 10, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}


### Conclusão do modelo


Após estimar o algoritmo com melhor acurácia para a tarefa em questão, montei manualmente o modelo utilizando esse algoritmo e com a base de treino padronizada. A técnica de padronização foi escolhida frente a de normalização e de ausência de tratamento pois apresentou melhor resultado.<br />
Apliquei o mesmo padrão aos dados de teste e fiz a predição desses dados pelo modelo. A pontuação de acurácia obtida reflete a performance do modelo ao prever os dados de teste em comparação com seus valores reais.<br />
Por último, toda a base de dados foi padronizada (com um novo ajuste do scaler) e o modelo foi re-treinado com todos os dados disponíveis.

In [ ]:
scaler = StandardScaler().fit(X_train)
scaledTrainX = scaler.transform(X_train)
scaledTestX = scaler.transform(X_test)

model = SVC(kernel='rbf', C=10, gamma=0.01)
model.fit(scaledTrainX, y_train)
predictions = model.predict(scaledTestX)
print("%.2f%%" % (accuracy_score(y_test, predictions) * 100))

scaler = StandardScaler().fit(X)
scaledX = scaler.transform(X)
model.fit(scaledX, y)

83.24%


SVC(C=10, gamma=0.01)

###Exportação do modelo, *encoders*, *scalers* e arquivos

In [ ]:
preprocessor = {
    "age_medians": age_medians,
    "age_medians_overall": age_medians_overall,
    "embarked_mode_pclass1": embarked_mode_pclass1,
    "sex_encoder": le,
    "embarked_cols": [col for col in df.columns if col.startswith('Embarked')],
    "scaler": scaler
}

with open('titanic_model_bundle.pkl', 'wb') as f:
    pickle.dump({
        "model": model,
        "preprocessor": preprocessor
    }, f)

y_test_df = y_test.to_frame()
test_df = pd.concat([X_test, y_test_df], axis=1)
test_df.to_csv("test_dataset_titanic.csv", index=False)

##Conclusão

O desenvolvimento desta tarefa, aliado ao conhecimento adquirido nas disciplinas estudadas,  foi fundamental para o aprofundamento do aprendizado e compreensão do processo de resolução de um problema de classificação. Os principais pontos de aprendizado foram:
*   Antes de iniciar qualquer codificação, foi importante realizar um estudo prévio dos dados e de seus atributos, compreendendo seus significados, identificando possíveis correlações e formulando hipóteses iniciais (por exemplo, pessoas portando tickets de classes mais altas teriam maior chance de sobrevivência do que aquelas portadoras de tickets de classe baixa).
*   A necessidade de tratar os atributos para adequar os dados ao formato exigido pelos algoritmos do **scikit-learn**, e como lidar com o preenchimento de dados ausentes, exigiu um nível de compreensão do problema. Essa etapa não pôde ser feita de maneira arbitrária, tendo sido necessário interpretar o contexto dos dados para garantir que os ajustes fossem coerentes, algo que foi facilitado pela análise inicial.
*   Existem algoritmos que são consideravelmente mais sensíveis ao redimensionamento (normalização e padronização) dos dados, o que impacta diretamente em seus desempenhos.
*   A busca pela melhor combinação de hiperparâmetros pode demandar um custo computacional elevado, especialmente técnicas como *grid search* em modelos mais complexos.

A seguir, apresento uma análise dos resultados obtidos pelos algoritmos, com ênfase na comparação das médias de acurácia antes e após a aplicação de técnicas de pré-processamento:

|Algoritmo       | Pré-processamento | Média Original | Média Pós | Melhora (%) |
| --------------- | ----------------- | -------------- | --------- | ----------- |
| **SVM**         | Padronização      | 0.6700         | 0.8062    | +20,33% |
| **SVM**         | Normalização      | 0.6700         | 0.7964    | +18,87%     |
| **KNN**         | Padronização      | 0.7078         | 0.7922    | +11,92% |
| **KNN**         | Normalização      | 0.7078         | 0.7880    | +11,33%     |
| **CART**        | Padronização      | 0.7514         | 0.7654    | +1,86%  |
| **CART**        | Normalização      | 0.7514         | 0.7612    | +1,30%      |
| **Naive Bayes** | Todos             | 0.7781         | 0.7781    | 0%   |

Os resultados obtidos confirmam a teoria: algoritmos baseados em distância, como o SVM e o KNN, demonstraram-se bastante sensíveis ao redimensionamento dos dados, com melhorias significativas nas médias de acurácia. <br />
Por outro lado, o algoritmo baseado em árvore (CART) teve apenas uma melhora marginal com o redimensionamento. Já o Naive Bayes, que opera com base em probabilidades condicionais, mostrou-se totalmente insensível ao pré-processamento neste caso, mantendo a mesma média de acurácia independentemente da técnica aplicada.
Após essa análise, selecionei o algoritmo com melhor desempenho (SVM com padronização) para ajuste fino de hiperparâmetros. Inicialmente, explorei um conjunto abrangente de combinações (400 no total), obtendo os seguintes resultados:

|Algoritmo       | Pré-processamento | Média Original | Média Pós | Melhora (%) | C | gamma | kernel |
| --------------- | ----------------- | -------------- | --------- | ----------- | --------- | --------- | --------- |
| **SVM**         | Padronização      | 0.8062         | 0.8147    | +1,05% | 10 | 0.01 | rbf |
| **SVM**         | Normalização      | 0.7964         | 0.8048    | +1,05%  | 10 | 1 | rbf |
| **SVM**         | Nenhum      | 0.6700         | 0.7907    | +18,01% | 1 | scale | linear |

Observou-se que a aplicação de hiperparâmetros pode, em certos casos, mitigar parcialmente a ausência de um bom pré-processamento. Por exemplo, o uso de um gamma menor suavizou o impacto de variações na escala, enquanto um valor mais alto de C permitiu ao modelo capturar padrões mais complexos. No entanto, a combinação entre pré-processamento adequado e ajuste fino de parâmetros resultou no melhor desempenho geral.
O algoritmo SVM demonstrou ser particularmente sensível ao ajuste de hiperparâmetros, já que o parâmetro C influencia o equilíbrio entre viés e variância do modelo, enquanto gamma afeta a complexidade da fronteira de decisão — definindo o quanto cada amostra influencia o formato da superfície de separação.

Após a realização de diversas etapas — incluindo o tratamento dos dados, o pré-processamento e o ajuste de hiperparâmetros — foi possível treinar um modelo com média de acurácia em torno de 81%, demonstrando boa capacidade de lidar com o problema de classificação do Titanic.